### This Jupyter file is structured as:
* Problem 2 - Question 2a-3a
* Problem 2 - Question 2b-3b
* Problem 2 - Question 2c-3c
* Problem 2 - Question 2d-3d
* Results and Comments for both Questions

## Global Imports

In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import scipy.stats
from sklearn.metrics import mean_squared_error
from numpy.core.defchararray import mod
from sklearn.model_selection import KFold 
import math 
from numpy.ma.core import power
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score


## Functions used in Problems 2,3 - Questions 2a, 3a, 2b, 3b, 2c, 3c, 2d, 3d  

In [48]:
def calculate_AIC(k, L):
  """
    AIC criterion for selecting a model. 
    For the Pima Indians Diabetes Database: n=768 samples with k=8 features -> n/k=96 and we don't need to choose the second order criterion.
  """
  aic = (-2) * L + 2 * k

  return aic

In [49]:
def calculate_BIC(k, L, N):
  """
    BIC criterion for criterion for selecting a model based on information theory set within a Bayesian context. 
  """
  bic = (-2) * L + k * np.log(N)

  return bic

In [50]:
# Vectors collecting the results of the AIC, BIC criterion of classes 0 and 1 on entire dataset + collecting 
# average accuracies from Bayes classifiers.
AIC_0 = []
AIC_1 = []
BIC_0 = []
BIC_1 = []
average_accuracy = []

## Dataset upload and preprocessing

In [51]:
# Dataset preprocessing
df = pd.read_csv("./UCIdata-exercise1/pima-indians-diabetes.data", header= None)

np_A = df.values
np.take(np_A,np.random.permutation(np_A.shape[0]),axis=0,out=np_A)

np_X = np_A[:, 0:8]  
Y = np_A[:, [8]]
Y = np.array(Y, dtype=np.int64)

X_0 = []
X_1 = []


for i, value in enumerate(np_X):
  if(Y[i] == 0):
    X_0.append(value)
  elif(Y[i] == 1):
    X_1.append(value)

X_0 = np.array(X_0, dtype=np.float64)
X_1 = np.array(X_1, dtype=np.float64)

## Functions used in Problem 2 - Question 2a

In [52]:
def calculate_theta_2a(X):
  """
   Calculates mean value theta_ML of the distribution by the equation described in the next section.
  """
  x1 = 0

  for i in range(len(X)):
    x1 = x1 + X[i]
  
  theta = (1 / len(X)) * x1
  theta = np.reshape(theta,(len(X[0]),1))
  return theta

In [53]:
def calculate_a_2a(X, theta):
  """
   Calculates variance a of covariance matrix Sigma = a*identity of the distribution by the equation described in the next section..
  """
  x3 = 0

  for i in range(len(X)):
    x1 = X[i] - theta.T
    x2 = np.matmul(x1, x1.T)
    x3 = x3 + x2
  
  a = (2 / (len(X) * len(X[0]))) * x3
  return a

In [54]:
def calculate_log_likelihood_2a(X, theta, a):
  """
    Calculates log likelihood of the distribution given the assumptions we made about its form.
  """
  x1 = (-(len(X) * len(X[0])) / 2) * np.log(2 * math.pi)
  x2 = (-(len(X) * len(X[0]))/4) * np.log(a)
  x3 = 0

  for i in range(len(X)):
    x4 = X[i] - theta.T
    x5 = np.matmul(x4, x4.T)
    x3 = x3 + x5

  x3 = (-1/(2 * a)) * x3

  L = x1 + x2 + x3

  return L

## Problem 2 - Question 2a

What we want to do is obtain an estimate for the probability density functions (pdfs) of our 2 classes. 

What we have is our training set (where we consider all samples independent one from another), and the assumption that the distribution of our pdf's is guassian, with covariance matrices that are diagonal, and all of those diagonal elements are equal. So we consider that $p(\vec{x})$ is known, and we want to find estimates for the unknown parameter vector $\vec{θ}$ and the variance.

$p(\vec{x}) ≡ p(\vec{x}|\vec{θ})$, and so the likelihood of $\vec{\theta}$ with respect to $X$ is:

 $p(X|\vec{θ}) ≡ p(\vec{x_{1}},...,\vec{x_{N}}|\vec{\theta}) ≡ p(\vec{x}|{\theta}) = \prod_{k=1}^N p(\vec{x_{k}}|\vec{\theta})$

In order to find our estimates, we are going to implement the Maximum Likelihood method (ML), according to which we calculate them by maximizing the log likelihood with respect to $\vec{\theta}$ and $Σ$.

$p(\vec{x_k})=\frac{1}{(2\pi)^{\frac{n}{2}}|\Sigma|^{\frac{1}{2}}}exp(-\frac{1}{2}(\vec{x_k}-\vec{\theta})^T\Sigma^{-1}(\vec{x_k}-\vec{\theta}))=p(\vec{x_k}|\vec{\theta},\Sigma)$

Log likelihood: $\Pi(\vec{\theta},\Sigma)=ln(\prod_{k=1}^N p(\vec{x_{k}}|\vec{\theta}))=
-Nln((2\pi)^{\frac{n}{2}})-Nln(|\Sigma|^{\frac{1}{2}})-\frac{1}{2}\sum_{k=1}^N(\vec{x_k}-\vec{\theta})^T\Sigma^{-1}(\vec{x_k}-\vec{\theta})$

Here $\Sigma=aI$ and $det(I)=1 \rightarrow det(aI)=a^n$ which leads to $|\Sigma|^{\frac{1}{2}}=a^{\frac{n}{2}}$ and $\Sigma^{-1}\Sigma=\Sigma^{-1}aI \rightarrow \Sigma^{-1}=\frac{1}{a}I$.


After substituting $\Sigma^{-1}$ the log-likelihood becomes: $\Pi(\vec{\theta},aI)=-Nln((2\pi)^{\frac{n}{2}})-Nln((a^{\frac{n}{2}})^{\frac{1}{2}})-\frac{1}{2}\sum_{k=1}^N(\vec{x_k}-\vec{\theta})^T\frac{1}{a}I(\vec{x_k}-\vec{\theta})$

Afterwards, we take the partial derivatives with respect to $a$ and $\vec{\theta}$:
* $\frac{\partial \Pi}{\partial a}=0 
\rightarrow 0 - \frac{Nn}{4}\frac{1}{a}+\frac{1}{2a^2}\sum_{k=1}^N(\vec{x_k}-\vec{\theta})^TI(\vec{x_k}-\vec{\theta}) = 0 \rightarrow  \frac{Nn}{4}\frac{1}{a}+\frac{1}{2a^2} =\sum_{k=1}^N(\vec{x_k}-\vec{\theta})^TI(\vec{x_k}-\vec{\theta}) 
\rightarrow a=\frac{2}{Nn}\sum_{k=1}^N(\vec{x_k}-\vec{\theta})^TI(\vec{x_k}-\vec{\theta}) \rightarrow
a=\frac{2}{Nn}\sum_{k=1}^N(\vec{x_k}-\vec{\theta})^2$

* $\frac{\partial \Pi}{\partial \vec{\theta}} = 0 \rightarrow
\sum_{k=1}^N(\vec{x_k}-\vec{\theta})=0 \rightarrow 
\sum_{k=1}^N\vec{x_k}=N\vec{\theta} \rightarrow \vec{\theta_{ML}}=\frac{1}{N}\sum_{k=1}^N\vec{x_k}$

So we have splitted our whole dataset in two parts, according to the class each of the samples belongs to, and we are going to plug those 2 sub-datasets into the equations we derived, in order to obtain our calculations for the estimates of the mean and the variance of the 2 pdfs:

In [55]:
# Estimates for class 0
theta_ml_0 = calculate_theta_2a(X_0)
a_0 = calculate_a_2a(X_0, theta_ml_0)

print(f"Estimation of mean for class 0: \n{theta_ml_0}")
print(f"Estimation of variance for class 0: {a_0}")

Estimation of mean for class 0: 
[[  3.298   ]
 [109.98    ]
 [ 68.184   ]
 [ 19.664   ]
 [ 68.792   ]
 [ 30.3042  ]
 [  0.429734]
 [ 31.19    ]]
Estimation of variance for class 0: [[2796.93338887]]


In [56]:
# Estimates for class 1
theta_ml_1= calculate_theta_2a(X_1)
a_1 = calculate_a_2a(X_1, theta_ml_1)

print(f"Estimation of mean for class 1:\n {theta_ml_1}")
print(f"Estimation of variance for class 1: {a_1}")

Estimation of mean for class 1:
 [[  4.86567164]
 [141.25746269]
 [ 70.82462687]
 [ 22.1641791 ]
 [100.3358209 ]
 [ 35.14253731]
 [  0.5505    ]
 [ 37.06716418]]
Estimation of variance for class 1: [[5284.32644968]]


In order to assess the goodness of our models for each class, we are going to  calculate the (AIC) and the (BIC) criterions:

$(AIC) = -2log(L(\vec{Θ})) + 2k$, 

where: $L(\theta)$ is the  likelihood  of  the  candidate  model  given  the  data  when  evaluated at the maximum likelihood estimate of $\vec{θ}$, and $k$ the number of estimated parameters in the candidate model (2 in our cases).
So, a model with a high value for the likelihhod function scores a low AIC.
Models with large numbers of parameters tend to score high values of the $L(\vec{\theta}) $function, but theres a drawback to that: lots of parameters increase the complexity of the model. And that exactly is depicted by the term "$+2k$", that is used to penalise such models. 

Also, it is kind of pointless to calculate AIC only for one model. AIC is calculated when we want to compare models, and the “best” model is the candidate with the smallest AIC. 

As for the BIC criterion, it is calculated by the formula:
$(BIC) = -2log(L(\vec{\theta})) + klogn$

where $n$ is the number of observations.

The difference between BIC and the AIC is the greater penalty imposed on the term that has to do with the number of parameters.

In [57]:
# Finding the log likelihoods of each class and calculating AIC and BIC criterion.
k = 2
log_likelihood_1 = calculate_log_likelihood_2a(X_1, theta_ml_1, a_1)

log_likelihood_0 = calculate_log_likelihood_2a(X_0, theta_ml_0, a_0)

aic_1 = calculate_AIC(k, log_likelihood_1)
bic_1 = calculate_BIC(k, log_likelihood_1, len(X_1))

aic_0 = calculate_AIC(k, log_likelihood_0)
bic_0 = calculate_BIC(k, log_likelihood_0, len(X_0))

AIC_0.append(aic_0)
AIC_1.append(aic_1)
BIC_0.append(bic_0)
BIC_1.append(bic_1)

## Functions used in Problem 3 - Question 3a

In [58]:
def bayes_classifier_3a(x, th_1, th_0):
  """
    Classifies points x to classes: 1 (with its distribution having mean th_1) and 0 (with its distribution having mean th_0).
    The classification is done by calculating the Euclidean distance ||x-mean_value|| between the point x and the classes.
  """
  # Class 1
  a = np.linalg.norm(th_1 - x)
  # Class 0
  b = np.linalg.norm(th_0 - x)

  if a > b:
    # Euclidean distance of x is smaller to class 0 than to class 1.
    return 0
  else:
    # Euclidean distance of x is smaller to class 1 than to class 0.
    return 1



In [59]:
def produce_y_pred_3a(X_tst, th_1, th_0):
    """
     Performs classification using the Bayes classifier to the test set and collects the
     results to a vector.
    """
    Y_pred = []

    for i in range(len(X_tst)):
      Y_pred.append(bayes_classifier_3a(X_tst[i], th_1, th_0))

    return Y_pred

In [60]:
def cross_validation_3a(k):
  """
    Performs k-fold validation in the dataset, finding the average error between all folds in order to give a not biased view 
    of the performance of the classifier.
  """
  accu = []

  kf = KFold(n_splits=k, random_state=None)
  
  for i , j in kf.split(np_X):
    X_train , X_test = np_X[i,:], np_X[j,:]
    y_train , y_test = Y[i], Y[j]

    X_1 = []
    X_0 = []

    for i, value in enumerate(X_train):
      if y_train[i] == 1:
        X_1.append(value)
      else:
        X_0.append(value)

    theta_ml_1= calculate_theta_2a(X_1)

    theta_ml_0= calculate_theta_2a(X_0)

    y_pred = produce_y_pred_3a(X_test, theta_ml_1, theta_ml_0)
    
    acc = accuracy_score(y_test, y_pred)
    accu.append(acc)
     
  avg_acc = sum(accu)/k
  
  return avg_acc

## Problem 3 - Question 3a

A Bayes classifier assigns $\vec{x}$ to $\omega_1$ if:
 $P(\omega_1|\vec{x})>P(\omega_2|\vec{x}) $ 
 and to $\omega_2$ if $P(\omega_1|\vec{x}) < P(\omega_2|\vec{x}) $.

For equiprobable classes the test becomes: $p(\vec{x}|\omega_1)>p(\vec{x}|\omega_2)$ and $p(\vec{x}|\omega_1)< p(\vec{x}|\omega_2)$ respectively.

* In the case of $\Sigma=\sigma^2I$, (according to the calculations we did in the lectures), we concluded that we can assign class $\omega_1$ if the Euclidean distance of $\vec{x}$ to $\omega_1$ is smaller than the respective distance to $\omega_2$:
  $\vec{x} \rightarrow \omega_1: ||\vec{x}-\vec{\mu_{1}}||<||\vec{x}-\vec{\mu_{2}}||$

In this question, the Bayes classifier we designed calculates the distances between the query vector we want to classify, and the estimations of the means of the pdfs that correspond to each one of our classes, which we have calculated using the ML method.

In order to calculate the accuracy of our classifier, we are going to implement K-fold cross validation for K = 10. As we have mentioned before, at each iteration of the method, we split our dataset into train and test sets. Now, we use the train set in order to estimate the parameters of each of the pdfs of our classes, and the test set to test the estimated model. 


In [61]:
#obtain average accuracy from Kfold (for K = 10)
avg_acc_3a = cross_validation_3a(10)

average_accuracy.append(avg_acc_3a)

## Functions used in Problem 2 - Question 2b

In [62]:
def calculate_theta_2b(X):
  """
   Calculates mean value theta_ML of the distribution by the equation described in the next section.
  """
  x1 = 0

  for i in range(len(X)):
    x1 = x1 + X[i]
  
  theta = (1 / len(X)) * x1
  theta = np.reshape(theta,((len(X[0]),1)))
  return theta

In [63]:
def calculate_covariance_2b(X, theta):
  """
   Calculates covariance matrix of the distribution by the equation described in the next section.
  """
  x3 = 0

  for i in range(len(X)):
    x1 = X[i] - theta.T
    x2 = np.matmul(x1.T, x1)
    x3 = x3 + x2
  
  a = (1 / len(X)) * x3
  return a

In [64]:
def calculate_log_likelihood_2b(X, theta, a):
  """
   Calculates log likelihood by the equation described in the next section.
  """
  x1 = (-(len(X) * len(X[0])) / 2) * np.log(2 * math.pi)
  x2 = (-len(X) / 2) * np.log(np.linalg.det(a))
  x3 = 0

  for i in range(len(X)):
    x4 = X[i] - theta.T
    x5 = np.matmul(x4, np.linalg.inv(a))
    x6 = np.matmul(x5, x4.T)
    x3 = x3 + x6

  x3 = (-1/2) * x3

  L = x1 + x2 + x3

  return L

## Problem 2 - Question 2b

Again, what we want to do is obtain an estimate for the probability density functions (pdfs) of our 2 classes. We have our training set, and now we are making the assumption that the distribution of our pdf's is guassian, but with  covariance matrices that are not diagonal. In order to find our estimates, we are going to implement the Maximum Likelihood method (ML) once again, so we are going to according to maximize the log likelihood with respect to $\vec{\theta}$ and $Σ$: 




The Log likelihood is equal to: $\Pi(\vec{\theta},\Sigma)=ln(\prod_{k=1}^N p(\vec{x_{k}}|\vec{\theta}))=
-Nln((2\pi)^{\frac{n}{2}})-Nln(|\Sigma|^{\frac{1}{2}})-\frac{1}{2}\sum_{k=1}^N(\vec{x_k}-\vec{\theta})^T\Sigma^{-1}(\vec{x_k}-\vec{\theta})$.



Using the property $trace(ba^T)=a^Tb$ with $a, b$ real valued matrices we obtain: $x^TAx=x^T(Ax)=trace(A^Tx^Tx)=trace(Axx^T)$ with $A$ a symmetric matrix. 


The Log likelihood becomes:

 $\Pi(\vec{\theta},\Sigma)=
-Nln((2\pi)^{\frac{n}{2}})+\frac{1}{2}Nln(|\Sigma|^{-1})-\frac{1}{2}\sum_{k=1}^Ntrace[(\vec{x_k}-\vec{\theta})(\vec{x_k}-\vec{\theta})^T\Sigma^{-1}]
=
-Nln((2\pi)^{\frac{n}{2}})+\frac{1}{2}Nln(|\Sigma|^{-1})-\frac{1}{2}trace[\sum_{k=1}^N(\vec{x_k}-\vec{\theta})(\vec{x_k}-\vec{\theta})^T\Sigma^{-1}]$


By taking the same partial derivatives equal to 0 as before:


* $\frac{\partial \Pi}{\partial \Sigma^{-1}} = 0 \rightarrow \frac{N}{2}((\Sigma^{-1})^{-1})^T-\frac{1}{2}\sum_{k=1}^N(\vec{x_k}-\vec{\theta})(\vec{x_k}-\vec{\theta})^T = 0 \rightarrow
\Sigma = \frac{1}{N}\sum_{k=1}^N(\vec{x_k}-\vec{\theta})(\vec{x_k}-\vec{\theta})^T$


* $\frac{\partial \Pi}{\partial \vec{\theta}}=0 \rightarrow ... \rightarrow \vec{\theta_{ML}}=\frac{1}{N}\sum_{k=1}^N\vec{x_k}$

The equations were adapted from: https://groups.seas.harvard.edu/courses/cs281/files/section01.pdf?fbclid=IwAR2Irz2_entsrh5nGDYAV9F2EkoZUVQdm3by-Y01oKp5yR3svs7kLtRxI4U

In [65]:
# Estimates for class 0
theta_ml_0 = calculate_theta_2b(X_0)
a_0 = calculate_covariance_2b(X_0, theta_ml_0)

print(f"Estimation of mean for class 0:\n {theta_ml_0}")
print(f"Estimation of covariance matrix for class 0:\n {a_0}")

Estimation of mean for class 0:
 [[  3.298   ]
 [109.98    ]
 [ 68.184   ]
 [ 19.664   ]
 [ 68.792   ]
 [ 30.3042  ]
 [  0.429734]
 [ 31.19    ]]
Estimation of covariance matrix for class 0:
 [[ 9.08519600e+00  7.76796000e+00  7.23916800e+00 -5.30587200e+00
  -3.92920160e+01  3.81948400e-01 -7.20027320e-02  2.01233800e+01]
 [ 7.76796000e+00  6.81995600e+02  9.08536800e+01  6.22128000e+00
   9.10377840e+02  2.64314840e+01  7.45542680e-01  6.94078000e+01]
 [ 7.23916800e+00  9.08536800e+01  3.25622144e+02  5.02138240e+01
   1.33002272e+02  5.03454272e+01  1.47144944e-01  4.51570400e+01]
 [-5.30587200e+00  6.22128000e+00  5.02138240e+01  2.21267104e+02
   6.06452112e+02  5.01206112e+01  4.23028624e-01 -2.83981600e+01]
 [-3.92920160e+01  9.10377840e+02  1.33002272e+02  6.06452112e+02
   9.75479674e+03  1.92872674e+02  6.71014467e+00 -1.71800480e+02]
 [ 3.81948400e-01  2.64314840e+01  5.03454272e+01  5.01206112e+01
   1.92872674e+02  5.90156024e+01  1.62197517e-01  3.22980200e+00]
 [-7.20027

In [66]:
# Estimates for class 1
theta_ml_1= calculate_theta_2b(X_1)
a_1 = calculate_covariance_2b(X_1, theta_ml_1)

print(f"Estimation of mean for class 1:\n {theta_ml_1}")
print(f"Estimation of covariance matrix for class 1:\n {a_1}")

Estimation of mean for class 1:
 [[  4.86567164]
 [141.25746269]
 [ 70.82462687]
 [ 22.1641791 ]
 [100.3358209 ]
 [ 35.14253731]
 [  0.5505    ]
 [ 37.06716418]]
Estimation of covariance matrix for class 1:
 [[ 1.39446425e+01 -6.49899755e+00  1.01704723e+01 -5.21675206e+00
  -4.06116061e+01 -4.30622633e+00 -9.60335821e-02  1.81918579e+01]
 [-6.49899755e+00  1.01633297e+03  4.69817192e+01  2.11629539e+01
   1.15345458e+03  1.16521079e+01  3.13677239e-01  3.44006182e+01]
 [ 1.01704723e+01  4.69817192e+01  4.60174468e+02  8.52675986e+01
   2.65379789e+02  2.08309674e+01  2.75236940e-01  6.16908833e+01]
 [-5.21675206e+00  2.11629539e+01  8.52675986e+01  3.11405881e+02
   1.11529561e+03  3.99210013e+01  1.79638806e+00 -1.77759523e+01]
 [-4.06116061e+01  1.15345458e+03  2.65379789e+02  1.11529561e+03
   1.91629021e+04  5.53069837e+01  5.22539925e+00  3.62871464e+01]
 [-4.30622633e+00  1.16521079e+01  2.08309674e+01  3.99210013e+01
   5.53069837e+01  5.25538622e+01  3.68475000e-01 -1.49215137

In [67]:
# Finding the log likelihoods of each class and calculating AIC and BIC criterion.
k = 2

log_likelihood_1 = calculate_log_likelihood_2b(X_1, theta_ml_1, a_1)

log_likelihood_0 = calculate_log_likelihood_2b(X_0, theta_ml_0, a_0)

aic_1 = calculate_AIC(k, log_likelihood_1)
bic_1 = calculate_BIC(k, log_likelihood_1, len(X_1))

aic_0 = calculate_AIC(k, log_likelihood_0)
bic_0 = calculate_BIC(k, log_likelihood_0, len(X_0))


AIC_0.append(aic_0)
AIC_1.append(aic_1)
BIC_0.append(bic_0)
BIC_1.append(bic_1)

## Functions used in Problem 3 - Question 3a

In [68]:
def bayes_classifier_3b(x, th_1, th_0, a_1, a_0):
  """
    Classifies points x to classes: 1 (with its distribution having mean th_1 and cov.matrix a_1) and 0 (with its distribution having mean th_0 and cov.matrix a_0).
    The classification is done by calculating the Mahalanobis distance [(x-mean_value)^T cov.matrix^{-1} (x-mean_value)] between the point x and the classes.
  """
  x = np.reshape(x,(1,8))
  x1 = np.matmul((x - th_1.T), np.linalg.inv(a_1))
  a = np.matmul(x1, (x - th_1.T).T)

  x3 = np.matmul((x - th_0.T), np.linalg.inv(a_0))
  b = np.matmul(x3, (x - th_0.T).T)
  
  if a > b:
    # Classify point to class 0.
    return 0
  else:
    # Classify point to class 1.
    return 1



In [69]:
def produce_y_pred_3b(X_tst, th_1, th_0, a_1, a_0):
    """
        Performs classification using the Bayes classifier to the test set and collects the
        results to a vector.
    """
    Y_pred = []

    for i in range(len(X_tst)):
      Y_pred.append(bayes_classifier_3b(X_tst[i], th_1, th_0, a_1, a_0))

    return Y_pred

In [70]:
def cross_validation_3b(k):
  """
    Performs k-fold validation in the dataset, finding the average error between all folds in order to give a not biased view 
    of the performance of the classifier.
  """
  accu = []

  kf = KFold(n_splits=k, random_state=None)
  
  for i , j in kf.split(np_X):
    X_train , X_test = np_X[i,:], np_X[j,:]
    y_train , y_test = Y[i], Y[j]

    X_1 = []
    X_0 = []

    for i, value in enumerate(X_train):
      if y_train[i] == 1:
        X_1.append(value)
      else:
        X_0.append(value)

    theta_ml_1= calculate_theta_2b(X_1)
    a_1 = calculate_covariance_2b(X_1, theta_ml_1)


    theta_ml_0= calculate_theta_2b(X_0)
    a_0 = calculate_covariance_2b(X_0, theta_ml_0)


    y_pred = produce_y_pred_3b(X_test, theta_ml_1, theta_ml_0, a_1, a_0)
    
    acc = accuracy_score(y_test, y_pred)
    accu.append(acc)
     
  avg_accu = sum(accu)/k
  
  return avg_accu

## Problem 3 - Question 3b

In this question, we are dealing with the case where the covariance matrix is not diagonal ($\Sigma \neq \sigma^2 I$). According to calculations we performed in the lectures, we concluded that we can perform classification using the Mahalanobis distance: 

We assign $\vec{x}$ to $\omega_1$ if $(\vec{x}-\vec{\mu_1})^T\Sigma_1^{-1}(\vec{x}-\vec{\mu_1}) < (\vec{x}-\vec{\mu_2})^T\Sigma_2^{-1}(\vec{x}-\vec{\mu_2})$ (with the equivalent relationship for class $\omega_2$). 

So, the Bayes classifier we designed calculates the Mahalanobis distances between the query vector we want to classify, and the estimations of the means of the pdfs that correspond to each one of our classes, which we have calculated using the ML method.

In order to calculate the accuracy of our classifier, we are going to implement K-fold cross validation for K = 10. 

In [71]:
# Obtain average accuracy from Kfold (for K = 10)
avg_acc_3b = cross_validation_3b(10)

average_accuracy.append(avg_acc_3b)

## Functions used in Problem 2 - Questions 2c

In [72]:
def calculate_theta_2c(X):
  """
   Calculates vector thetas containing the mean value theta_j of each one of the l features of the dataset.
  """
  thetas = []

  for j in range(len(X[0])):
    x1 = 0
    
    for i in range(len(X)):
      x1 = x1 + X[i][j]
  
    thetas.append((1/len(X)) * x1)

  thetas = np.array(thetas, dtype=np.float64)
  thetas = np.reshape(thetas,(len(thetas),1))
  return thetas

In [73]:
def calculate_s_2c(X, theta):
  """
   Calculates covariances sigma_j^2 for the Gaussian distributions of each feature.
  """
  s = []
  x2 = 0

  for j in range(len(X[0])):
    x1 = 0
    x2 = 0

    for i in range(len(X)):
      x1 = X[i][j] - theta[j]
      x2 = x2 + np.power(x1, 2)
  
    s.append((1 / len(X)) * x2)
  
  s = np.array(s, dtype=np.float64)
  s = np.reshape(s,(len(s),1))
  return s

In [74]:
def calculate_log_likelihood_2c(X, theta, s):
  """
    Calculates log likelihood with the formulas derived in the next section.
  """
  L = 0 

  for i in range(len(X)):

    x1 = calculate_P_ol_2c(X[i], theta, s)
    x2 = math.log(x1)
    L = L + x2

  return L

In [75]:
def calculate_P_ol_2c(X, theta, s):
  """
    Calculates probability density function at a point X, given vectors containing the mean values and the covariances
    of each Gaussian distribution.
  """
  x1 = 1

  for i in range(len(X)):
    x2 = (1 / math.sqrt(2 * math.pi)) * (1 / s[i])
    x3 = -(np.power((X[i] - theta[i]), 2) / (2 * np.power(s[i], 2)))
    x4 = np.power(math.e, x3)
    
    x1 = x1 * x2 * x4

  return x1

## Problem 2 - Question 2c

Now we are making the assumption that the components of the vectors in our data set are mutually statistically independent. This allows us to do all calculations in seperate dimensions:

$p(\vec{x}|\vec{\theta}) = p(x_{1}|θ_{1})* ....* p(x_{\lambda}|θ_{\lambda})$

where: $\vec{x} = [x_{1},...,x_{\lambda}], \vec{θ} = [\theta_{1},...,\theta_{\lambda}]$, $\lambda$: number of features.

Also, we are assuming that the marginal pdf's are guassian:

For each feature $i$: $p(x_{i}|θ_{i})=\frac{1}{(2\pi)^{\frac{1}{2}} σ_{i}}exp(-\frac{(x_{i} - θ_{i})^2}{2σ_{i}^{2}})$

We are going to implement the ML method in order to find estimates for their means and variances. So we are implementing it in 1D, using each time only the components of a particular dimension from the vectors in our data set.
According to the ML method:

Log likelihood: $\Pi({\vec{\theta}},\vec{σ})=ln(\prod_{k=1}^N p(x_k|\theta))=ln(\prod_{k=1}^N (\prod_{i=1}^l \frac{1}{(2\pi)^{\frac{1}{2}} σ_{i}}exp(-\frac{(x_{ki} - θ_{i})^2}{2σ_{i}^{2}})))
=ln(\prod_{k=1}^N    [\frac{1}{(2\pi)^{\frac{1}{2}}}]^\lambda \frac{1}{\sigma_1 \cdot .. \sigma_\lambda} exp     (\sum_{i=1}^\lambda -\frac{(x_{ki} - θ_{i})^2}{2σ_{i}^{2}}))=
-Nln((2\pi)^{\frac{\lambda}{2}})-Nln(\prod_{i=1}^\lambda \sigma_i)-\sum_{k=1}^N \sum_{i=1}^\lambda \frac{(x_{ki}-\theta_i)^2}{2\sigma_i^2}$


We then take the partial derivatives with respect to $θ_{j}$ and $σ_{j}$ for every $j\epsilon[1,l]$:
* $\frac{\partial \Pi}{\partial σ_{j}}=0 
\rightarrow 0 - N\frac{1}{σ_{j}}-\frac{1}{2}(-2)\sum_{k=1}^N\frac{(x_{kj}-\theta_{j})^2}{σ_{j}^{3}} = 0 \rightarrow  \sum_{k=1}^N\frac{(x_{kj}-\theta_{j})^2}{σ_{j}^{3}} =  N\frac{1}{σ_{j}} \rightarrow σ_{j}^{2} =  \sum_{k=1}^N\frac{(x_{kj}-\theta_{j})^2}{N}$

* $\frac{\partial \Pi}{\partial \theta_j} = 0 \rightarrow \frac{1}{σ_j^{2}}
\sum_{k=1}^N(x_{kj}-{\theta_{j}}){}=0 \rightarrow \sum_{k=1}^N(x_{nj})= N\theta_j \rightarrow \theta_j = \frac{\sum_{k=1}^N(x_{nj})}{N} $


In [76]:
# Estimates for class 1
theta_ml_1= calculate_theta_2c(X_1)
s_1 = calculate_s_2c(X_1, theta_ml_1)

print(theta_ml_1)
print(s_1)

[[  4.86567164]
 [141.25746269]
 [ 70.82462687]
 [ 22.1641791 ]
 [100.3358209 ]
 [ 35.14253731]
 [  0.5505    ]
 [ 37.06716418]]
[[1.39446425e+01]
 [1.01633297e+03]
 [4.60174468e+02]
 [3.11405881e+02]
 [1.91629021e+04]
 [5.25538622e+01]
 [1.38130519e-01]
 [1.19853698e+02]]


In [77]:
# Estimates for class 0
theta_ml_0 = calculate_theta_2c(X_0)
s_0 = calculate_s_2c(X_0, theta_ml_0)

print(theta_ml_0)
print(s_0)

[[  3.298   ]
 [109.98    ]
 [ 68.184   ]
 [ 19.664   ]
 [ 68.792   ]
 [ 30.3042  ]
 [  0.429734]
 [ 31.19    ]]
[[9.08519600e+00]
 [6.81995600e+02]
 [3.25622144e+02]
 [2.21267104e+02]
 [9.75479674e+03]
 [5.90156024e+01]
 [8.92731152e-02]
 [1.35861900e+02]]


In [78]:
# Finding the log likelihoods of each class and calculating AIC and BIC criterion.
k = 2
log_likelihood_1 = calculate_log_likelihood_2c(X_1, theta_ml_1, s_1)

log_likelihood_0 = calculate_log_likelihood_2c(X_0, theta_ml_0, s_0)

aic_1 = calculate_AIC(k, log_likelihood_1)
bic_1 = calculate_BIC(k, log_likelihood_1, len(X_1))

aic_0 = calculate_AIC(k, log_likelihood_0)
bic_0 = calculate_BIC(k, log_likelihood_0, len(X_0))  

AIC_0.append(aic_0)
AIC_1.append(aic_1)
BIC_0.append(bic_0)
BIC_1.append(bic_1)

## Functions used in Problem 3 - Question 3c

In [79]:
def naive_bayes_classifier_3c(x, th_1, th_0, a_1, a_0):
  """
    Naive Bayes Classifier, classifies to w_1 if p(x|w_1)>p(x|w_2)
  """
  a = calculate_P_ol_2c(x , th_1, a_1)
  b = calculate_P_ol_2c(x, th_0, a_0)

  if a > b:
    # Classify to class 1.
    return 1
  else:
    # Classify to class 0.
    return 0

In [80]:
def produce_y_pred_3c(X_tst, th_1, th_0, a_1, a_0):
    """
        Performs classification using the Bayes classifier to the test set and collects the
        results to a vector.
    """
    Y_pred = []

    for i in range(len(X_tst)):
      Y_pred.append(naive_bayes_classifier_3c(X_tst[i], th_1, th_0, a_1, a_0))

    return Y_pred

In [81]:
def cross_validation_3c(k):
  """
    Performs k-fold validation in the dataset, finding the average error between all folds in order to give a not biased view 
    of the performance of the classifier.
  """
  accu = []

  kf = KFold(n_splits=k, random_state=None)
  
  for i , j in kf.split(np_X):
    X_train , X_test = np_X[i,:], np_X[j,:]
    y_train , y_test = Y[i], Y[j]

    X_1 = []
    X_0 = []

    for i, value in enumerate(X_train):
      if y_train[i] == 1:
        X_1.append(value)
      else:
        X_0.append(value)

    theta_ml_1= calculate_theta_2c(X_1)
    s_1 = calculate_s_2c(X_1, theta_ml_1)

    theta_ml_0= calculate_theta_2c(X_0)
    s_0 = calculate_s_2c(X_0, theta_ml_0)

    y_pred = produce_y_pred_3c(X_test, theta_ml_1, theta_ml_0, s_1, s_0)
    
    acc = accuracy_score(y_test, y_pred)
    accu.append(acc)
     
  avg_accu = sum(accu)/k
  
  return avg_accu

## Problem 3 - Question 3c

In order to decide which class we are going to assign to each of the samples of our test set, we are going to implement the rule we mentioned before (which holds assuming that P($ω_{1}$) = P($ω_{2}$)): 

assign x⃗  to ω1 if: $p(\vec{x}|\omega_1) > p(\vec{x}|\omega_2)$ and to ω2 if $p(\vec{x}|\omega_1) < p(\vec{x}|\omega_2)$.

We have made the assumption that the components of the feature vectors are mutually statistically independent, and so each one of our pdf's are the products of marginal 1d pdf's, which are gaussian and whose parameters we calculate using our training set and implementing the ML method. In this case, we say that we are implementing a Naive Bayes classifier.

In order to calculate the accuracy of our classifier, once again we are going to implement K-fold cross validation for K = 10. 


In [82]:
# Obtain average accuracy from Kfold (for K = 10)
avg_acc_3c = cross_validation_3c(10)

average_accuracy.append(avg_acc_3c)

## Functions used in Problem 2 - Questions 2d

In [83]:
def kernel_2d(x):
  """
    Kernel of a 2-D Gaussian distribution N(0,1) : mean value 0 and variance 1
  """
  x1 = (1 / math.sqrt(2 * math.pi))
  x2 = -(np.power(x, 2) / 2)
  x3 = np.power(math.e, x2)
    
  x4 = x1 * x3

  return x4

In [84]:
def calculate_p_2d(x, X, l, h):
  """
    Calculates probability distribution for one feature using the equations derived in the following section. 
  """
  x1 = 1 / (len(X) * h)
  x2 = 0

  for i in range(len(X)):
    x3 =  (X[i][l] - x) / h 
    y = kernel_2d(x3)

    x2 = x2 + y
  
  p = x1 * x2

  return p

In [85]:
def calculate_P_ol_2d(x, X, h):
  """
    Calculates probability distribution for all features using the equations derived in the following section. 
  """
  x1 = 1

  for i in range(len(x)):
    x2 = calculate_p_2d(x[i], X, i, h)
    
    x1 = x1 * x2

  return x1

## Problem 2 - Question 2d

For once again we assume that the components of the feature vectors are mutually statistically independent:

$p(\vec{x}) = p(x_{1})* ....* p(x_{\lambda})$,

where: $\vec{x} = [x_{1},...,x_{λ}]$, λ: number of features.


Now, we are going to use only our data set, in order to directly estimate a function that we think will be a good aproximation to the above pdf. 
We will do this for each one of the marginal pdfs, using a method called 1d Parzen windows.
We will implement it with gaussian kernels, and we will take the width h of each window to be equal to the square root of the number of patterns in our data set. So:

$p(x_{l})$ = $\frac{1}{hN}$$\sum_{k=1}^Ng(\frac{x_{kl}-x_{l}}{h})$, $l\in[1,λ]$

We assume the mean and variance of our gaussian kernel to be 0 and 1 respectively, and so:

$g(x) = \frac{1}{(2\pi)^{\frac{1}{2}}}exp(-\frac{(x)^2}{2})$ $\rightarrow$ g($\frac{x_{kl}-x_{l}}{h}$) = $\frac{1}{(2\pi)^{\frac{1}{2}}}exp(-\frac{(x_{kl}-x_{l})^2}{2h^{2}})$

This results in:

$p(x_{l})$ = $\frac{1}{(2\pi)^{\frac{1}{2}}Νh}\sum_{k=1}^Nexp(-\frac{(x_{kl}-x_{l})^2}{2h^{2}}) = \frac{1}{Ν}\sum_{k=1}^Ng(x_{k})$

being the mean of N 1-d Gaussians of mean $x_{l}$ and variance $h^{2} = N$.

So, in overall, we get:

$p(\vec{x})$ = $\prod_{l=1}^{λ}\sum_{k=1}^N\frac{g(x_{n})}{N}$





##Comments
The parzen windows method is a non parametric method. We get an estimation for our pdf, not for parameters. So we didnt use the AIC and BIC criterions here.

## Functions used in Problem 3 - Question 3d

In [86]:
def naive_bayes_classifier_3d(x, X_1, X_0):
  """
    Naive Bayes Classifier, classifies to w_1 if p(x|w_1)>p(x|w_2)
  """
  a = calculate_P_ol_2d(x, X_1, np.sqrt(len(X_1)))

  b = calculate_P_ol_2d(x, X_0, np.sqrt(len(X_0)))

  if a > b:
    return 1
  else:
    return 0

In [87]:
def produce_y_pred_3d(X_tst, X_1, X_0):
    """
      Performs classification using the Bayes classifier to the test set and collects the
      results to a vector.
    """
    Y_pred = []

    for i in range(len(X_tst)):
      Y_pred.append(naive_bayes_classifier_3d(X_tst[i], X_1, X_0))

    return Y_pred

In [88]:
def cross_validation_3d(k):
  """
    Performs k-fold validation in the dataset, finding the average error between all folds in order to give a not biased view 
    of the performance of the classifier.
  """
  accu = []

  kf = KFold(n_splits=k, random_state=None)
  
  for i , j in kf.split(np_X):
    X_train , X_test = np_X[i,:], np_X[j,:]
    y_train , y_test = Y[i], Y[j]

    X_1 = []
    X_0 = []

    for i, value in enumerate(X_train):
      if y_train[i] == 1:
        X_1.append(value)
      else:
        X_0.append(value)

    y_pred = produce_y_pred_3d(X_test, X_1, X_0)
    
    acc = accuracy_score(y_test, y_pred)
    accu.append(acc)
     
  avg_accu = sum(accu)/k
  
  return avg_accu

## Problem 3 - Question 3d

In order to decide which class we are going to assign to each of the samples of our test set, we are going to implement the rule we mentioned before (which holds assuming that $P(ω_{1}) = P(ω_{2}))$: 

Assign $\vec{x}$  to $ω_1$ if: $p(\vec{x}|\omega_1) > p(\vec{x}|\omega_2)$ and to $ω_2$ if $p(\vec{x}|\omega_1) < p(\vec{x}|\omega_2)$.

We have made the assumption that the components of our feature vectors are mutually statistically independent, and so we do all calculations in seperate dimensions. So, again we are implementing a Naive Bayes classifier.

We split our data set, in training and test set, and we use the first in order to estimate a pdf for each one of our classes, using a method called 1d Parzen windows. Then we use our model to perform classification on the test set.

In order to calculate the accuracy of our classifier, we are going to implement K-fold cross validation for K = 10. 

In [89]:
# Obtain average accuracy from Kfold (for K = 10)
avg_acc_3d = cross_validation_3d(10)

average_accuracy.append(avg_acc_3d)

## Results and Comments for Question 2

In [90]:
# Helper function for cleaning the results
def cleanAIC_BIC(criterion):
  x = [criterion[0][0][0],
       criterion[1][0][0],
       criterion[2]]
  return x 

In [91]:
AIC_0_cl = cleanAIC_BIC(AIC_0)
AIC_1_cl = cleanAIC_BIC(AIC_1)
BIC_0_cl = cleanAIC_BIC(BIC_0)
BIC_1_cl = cleanAIC_BIC(BIC_1)

print(f"AIC criterion for class 0:\n{AIC_0_cl}")
print(f"AIC criterion for class 1:\n{AIC_1_cl}")
print(f"BIC criterion for class 0:\n{BIC_0_cl}")
print(f"BIC criterion for class 1:\n{BIC_1_cl}")

print(f"\n\nDeltas AIC of models b and c from a for class 0:\n{[AIC_0_cl[1]-AIC_0_cl[0], AIC_0_cl[2]-AIC_0_cl[0]]}")
print(f"Deltas AIC of models b and c from a for class 1:\n{[AIC_1_cl[1]-AIC_1_cl[0], AIC_1_cl[2]-AIC_1_cl[0]]}")

print(f"\n\nDeltas BIC of models b and c from a for class 0:\n{[BIC_0_cl[1]-BIC_0_cl[0], BIC_0_cl[2]-BIC_0_cl[0]]}")
print(f"Deltas BIC of models b and c from a for class 1:\n{[BIC_1_cl[1]-BIC_1_cl[0], BIC_1_cl[2]-BIC_1_cl[0]]}")

AIC criterion for class 0:
[25228.06602106212, 28492.823315258855, 48703.74336561089]
AIC criterion for class 1:
[14206.128906907672, 16027.863890855155, 26313.595061925662]
BIC criterion for class 0:
[25236.495237258965, 28501.2525314557, 48712.17258180774]
BIC criterion for class 1:
[14213.310880868694, 16035.045864816177, 26320.777035886684]


Deltas AIC of models b and c from a for class 0:
[3264.7572941967337, 23475.67734454877]
Deltas AIC of models b and c from a for class 1:
[1821.7349839474828, 12107.46615501799]


Deltas BIC of models b and c from a for class 0:
[3264.7572941967337, 23475.677344548774]
Deltas BIC of models b and c from a for class 1:
[1821.7349839474828, 12107.46615501799]


Examining the AIC Criterion, we can see that the (a) model outperforms the other 2 in both classes. If we calculate the delta AICs of the other two models from the optimal model (a), they end up extremely greater than 10, so they are surely not candidates for the best model to fit the distribution of the specific dataset.


For the BIC criterion, the best model is the one with the minimum BIC which is (again) the model (a). In this case, if we calculate the delta BICs of the other models with respect to the BIC* of the optimal model, they end up larger than 10. Taking into account the interpretation of the BIC criterion, there is very strong evidence that these candidate models are not the best model. 

## Results and Comments for Question 3

In [92]:
print("Average Accuracies from Bayes Classifiers:\n")
    
print(f"Bayes classifier from 3a: {average_accuracy[0]:.6f}")
print(f"Bayes classifier from 3b: {average_accuracy[1]:.6f}")
print(f"Bayes classifier from 3c: {average_accuracy[2]:.6f}")
print(f"Bayes classifier from 3d: {average_accuracy[3]:.6f}")

Average Accuracies from Bayes Classifiers:

Bayes classifier from 3a: 0.660304
Bayes classifier from 3b: 0.671770
Bayes classifier from 3c: 0.644651
Bayes classifier from 3d: 0.529938


From the average results of predicting results on a test set (with k-fold validation) on the specific dataset, we find that the classifier from 3b outperforms the other 3 (not with a large margin from 3a and 3c). That leads us to the conclusion that the assumptions we made in 3b were more fitting to this dataset than the other 3. 

Comparing the results from all the Bayes classifiers and the knn method, it is obvious that the knn method gives better results than the classifiers. Generally speaking, the Bayes classifiers (and especially the Naive classifiers) are a good stepping stone to begin making predictions on a dataset and cannot be considered the optimal way of classification.

However, we must make a special reference to the equiprobability of the two classes. The dataset is not split 50-50 between the two classes (500/268 samples for the two classes) so the property of equiprobability does not hold completely because it is most probable to chose class 0 than class 1 if we make a random selection from the dataset. 
